# Set-up

## Imports

In [1]:
# Standard
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
from tqdm.notebook import tqdm, trange
import plotly.graph_objects as go

# Sci-kit learn imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, average_precision_score, f1_score, classification_report, matthews_corrcoef, PrecisionRecallDisplay
import joblib

## Parameters

In [36]:
scope = 'BID'
input_path = f'../data/3_output/ML_dataset_{scope}.pkl'

start_month = 201901
end_month = 202212

subsample_size = None

target = 'Result'

features_to_encode = [
    'MargTech',
    'WorkDay',
    'Prov',
    'Tech',
]

other_features = [
    'hour',
    'SC_PC1',
    'SC_PC2',
    'IN_PC1',
    'IN_PC2',
    'CT_PC1',
    'CT_PC2',
    'PurchMGP',
    'SellMGP',
    'SolarAngle',
    'DeclAngle',
    'PVold',
    'PVnew',
    'PriceDiff',
]

features = other_features + features_to_encode

In [24]:
input_df = pd.read_pickle(input_path)
input_df

,hour,date,year,unit,scope,SC_PC1,SC_PC2,IN_PC1,IN_PC2,CT_PC1,...,SolarAngle,DeclAngle,WorkDay,Prov,Tech,PVold,PVnew,Price,PriceDiff,Result
2018010118UP_ALTOADDA_1_BID,18,20180101,2018,UP_ALTOADDA_1,BID,2.083112,0.480035,-1.932233,-0.782315,1.912031,...,76.773894,-23.011637,holiday,Milano,Hydro Run-of-river and poundage,0.095,0.095,26.22,-0.349,False
2018010119UP_ALTOADDA_1_BID,19,20180101,2018,UP_ALTOADDA_1,BID,1.249924,0.598779,-1.772487,-0.866850,0.891173,...,91.773894,-23.011637,holiday,Milano,Hydro Run-of-river and poundage,0.095,0.095,26.22,-0.329,False
2018010120UP_ALTOADDA_1_BID,20,20180101,2018,UP_ALTOADDA_1,BID,0.243493,0.768175,-1.758348,-0.872364,-0.334594,...,106.773894,-23.011637,holiday,Milano,Hydro Run-of-river and poundage,0.095,0.095,26.22,-0.324,False
2018010117UP_ARSIE_1_BID,17,20180101,2018,UP_ARSIE_1,BID,3.456008,0.661794,-2.372137,-0.624680,3.037220,...,61.773894,-23.011637,holiday,Belluno,Hydro Run-of-river and poundage,0.627,0.627,0.00,-1.000,False
2018010118UP_ARSIE_1_BID,18,20180101,2018,UP_ARSIE_1,BID,2.083112,0.480035,-1.932233,-0.782315,1.912031,...,76.773894,-23.011637,holiday,Belluno,Hydro Run-of-river and poundage,1.000,1.000,0.00,-1.000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022123117UP_VOGHERA_1_BID,17,20221231,2022,UP_VOGHERA_1,BID,0.237656,-2.008471,-1.806319,0.875068,-2.527147,...,61.887169,-23.085911,holiday,Pavia,Fossil Gas,0.540,0.540,135.00,-0.350,False
2022123118UP_VOGHERA_1_BID,18,20221231,2022,UP_VOGHERA_1,BID,0.055686,-2.259989,-1.329127,0.607693,-2.561474,...,76.887169,-23.085911,holiday,Pavia,Fossil Gas,1.000,1.000,136.00,-0.420,True
2022123119UP_VOGHERA_1_BID,19,20221231,2022,UP_VOGHERA_1,BID,-0.051126,-2.357014,-1.293692,0.548271,-2.484197,...,91.887169,-23.085911,holiday,Pavia,Fossil Gas,1.000,1.000,138.00,-0.414,True
2022123120UP_VOGHERA_1_BID,20,20221231,2022,UP_VOGHERA_1,BID,-0.029177,-2.330178,-1.385200,0.630366,-2.484197,...,106.887169,-23.085911,holiday,Pavia,Fossil Gas,1.000,1.000,141.00,-0.405,True


In [25]:

# We take of the fact that some categories could be absent in the training set but present in the test set
categories = [input_df[feature].unique() for feature in features_to_encode]

feature_transformer = make_column_transformer(
    (OneHotEncoder(categories=categories), features_to_encode),
    remainder="passthrough"
)

## Functions

In [26]:
def get_X_y(df):
    X = feature_transformer.fit_transform(df[features])
    y = df[target]
    return X, y

In [34]:
def predict_proba_monthly_recal(model, df):
    """
    For each observation of the dataset, if M is the corresponding month, outputs the predicted probability of the model when trained on the M-12 to M-1 period.
    We hence fit a number of models equal to the number of months in the dataset.
    This allows to test the the performance of the model in a "live" setting, where each month, the model is recalibrated with the new data.
    """
    X, y = get_X_y(df)
    
    observation_month = df.index.str[:6].astype(int)
    months = sorted(observation_month.unique())
    test_months = [month for month in  months if month >= start_month and month <= end_month]
    y_probs_list =  []

    for test_month in tqdm(test_months):
        # For every month M, we take the training period as M-12 to M-1
        idx = months.index(test_month)
        train_months = months[idx-12:idx]
        X_train, y_train = get_X_y(df[observation_month.isin(train_months)])
        # And the test period as month M
        X_test, y_test = get_X_y(df[observation_month == test_month])

        model.fit(X_train, y_train)

        y_probs = model.predict_proba(X_test)[:,1]

        APS = average_precision_score(y_test, y_probs)
        print('Average Precision Score over {:,} samples for month {} is: {}'.format(len(y_test), test_month, round(APS, 3)))
        print("\n")

        y_probs_list.append(y_probs)
    
    return pd.Series(np.concatenate(y_probs_list, axis=0), index=df[observation_month.isin(test_months)].index)

# Main

## Preprocessing

In [37]:
# Subsample
if subsample_size is not None:
    print(f'Subsampled {subsample_size} rows from the input dataset')
    df = input_df.copy()
    df['order'] = range(len(df))
    df = df.sample(subsample_size).sort_values('order')
    df.drop('order', axis=1, inplace=True)
else:
    df = input_df.copy()

## Model

# Run test with monthly recal

In [38]:
%%time

clf = RandomForestClassifier(
    random_state=42,
    n_jobs=-1
)

y_probs = predict_proba_monthly_recal(clf, df)


  0%|          | 0/48 [00:00<?, ?it/s]

Average Precision Score over 46,525 samples for month 201901 is: 0.664


Average Precision Score over 38,675 samples for month 201902 is: 0.695


Average Precision Score over 39,137 samples for month 201903 is: 0.763


Average Precision Score over 42,489 samples for month 201904 is: 0.73


Average Precision Score over 46,778 samples for month 201905 is: 0.742


Average Precision Score over 48,778 samples for month 201906 is: 0.585


Average Precision Score over 54,320 samples for month 201907 is: 0.589


Average Precision Score over 40,523 samples for month 201908 is: 0.633


Average Precision Score over 37,283 samples for month 201909 is: 0.642


Average Precision Score over 34,730 samples for month 201910 is: 0.725


Average Precision Score over 35,962 samples for month 201911 is: 0.812


Average Precision Score over 36,277 samples for month 201912 is: 0.868


Average Precision Score over 40,846 samples for month 202001 is: 0.797


Average Precision Score over 33,777 samples for mont

In [39]:
y_probs

2019010110UP_ALTOADDA_1_BID    0.010
2019010111UP_ALTOADDA_1_BID    0.010
2019010113UP_ALTOADDA_1_BID    0.010
2019010114UP_ALTOADDA_1_BID    0.020
2019010116UP_ALTOADDA_1_BID    0.020
                               ...  
2022123117UP_VOGHERA_1_BID     0.110
2022123118UP_VOGHERA_1_BID     0.533
2022123119UP_VOGHERA_1_BID     0.487
2022123120UP_VOGHERA_1_BID     0.452
2022123121UP_VOGHERA_1_BID     0.130
Length: 1646699, dtype: float64

In [42]:
y_probs.to_pickle(f'{scope}/model_predictions/predicted_probs_monthly_recal_rolling_12m.pkl')

---

# TESTS